In [1]:
import os
import requests
from datetime import date, datetime, timedelta
import pandas as pd

## Getting the Data

First, we check to see if recent (less than a week old) UserBenchmark data exists in the directory. If not, we download the latest csv files from source.

In [5]:
parts = ['CPU', 'GPU', 'SSD', 'RAM']

def get_filename(part):
    return f'{part}_UserBenchmarks.csv'

def update_data(parts):
    try:
        os.mkdir('data')
    except:
        pass
    
    os.chdir('data')
    
    for part in parts:
        download = False
        filename = get_filename(part)
        
        if os.path.exists(filename):
            timestamp = os.path.getmtime(filename)
            modified = datetime.fromtimestamp(timestamp)
            delta = datetime.today() - modified
            if delta > timedelta(days = 7):
                download = True
        else:
            download = True
            
        if download:
            url = f'https://www.userbenchmark.com/resources/download/csv/{filename}'
            response = requests.get(url)
            with open(filename, 'wb') as file:
                file.write(response.content)
    os.chdir('..')
update_data(parts)